# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX, MODELNET10_SOFA_TOILET_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
INDEX = MODELNET10_TOILET_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 128,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,  # 0.001 hits nan loss eventually
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 0.8,
                "kl_div_loss_weight": .1,
                "recon_loss_weight": 10000,
                "ll_weight": 1,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": True,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 5,
                "save_step": 5,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                #'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                #'tb_compare': [('vaegan_100epochs_toilets', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb')],
                #'tb_compare': [('vaegan_logless_loss', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-22_16-01-54/tb')],
                'tb_compare': [('voxel_vae_toilet_200epochs_1', os.path.join(MODEL_DIR, 'voxel_vae_toilet_200epochs_1', 'tb'))],
                'no_gan': True,
                'monitor_memory': True,
                # these settings control how often the components' optimizers are executed during the training loop
                'train_vae_cadence': 1,
                'train_gan_cadence': 1,
                'dis_noise': 0,
                'adaptive_lr': False
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b'22076\n'
killed b'22076'!


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [ ]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-25_11-05-00
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.adaptive_lr"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.dis_noise"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNI

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-25_11-05__root.log
Starting train_vaegan main
Numpy random seed: 459121366
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-25_11-05-00/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_toilet_index.csv and pctile None
Shuffling dataset
dataset n_input=7104
Splitting Datasets
Num input = 7104
Num batches per epoch = 55.50
Initializing VoxelVaegan
encoder/enc_conv1/batchnorm/add_1:0: (?, 30, 30, 30, 8)
encoder/batch_normalization/batchnorm/add_1:0: (?, 15, 15, 15, 16)
encoder/batch_normalization_1/batchnorm/add_1:0: (?, 13, 13, 13, 32)
encoder/batch_normalization_2/batchnorm/add_1:0: (?, 7, 7, 7, 64)
encoder/batch_normalization_3/batchnorm/add_1:0: (?, 7, 7, 7, 343)
encoder/dense_1/BiasAdd:0: (?, 100)
encoder/dense_2/BiasAdd:0: (?, 100)
epsilon: (?, 100)
z: (?, 100)
input_z: (?, 100)
decoder/dec_dense1/BiasAdd:

<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam_1:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/moving_mean/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normaliz

<tf.Variable 'decoder/dec_conv4/kernel/Adam_1:0' shape=(4, 4, 4, 8, 16) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam:0' shape=(3, 3, 3, 1, 8) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam_1:0' shape=(3, 3, 3, 1, 8) dtype=fl

Epoch: 1 / 201, Batch: 14 (1920 / 5504), Elapsed time: 3.62 mins
Enc Loss = 410.18, KL Divergence = 269.82, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 3.69 mins
Memory Use (GB): 1.3013839721679688
Epoch: 1 / 201, Batch: 15 (2048 / 5504), Elapsed time: 3.69 mins
Enc Loss = 405.35, KL Divergence = 232.18, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 3.75 mins
Memory Use (GB): 1.3165702819824219
Epoch: 1 / 201, Batch: 16 (2176 / 5504), Elapsed time: 3.75 mins
Enc Loss = 402.23, KL Divergence = 249.39, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 3.81 mins
Memory Use (GB): 1.3660469055175781
Epoch: 1 / 201, Batch: 17 (2304 / 5504), Elapsed time: 3.81 mins
Enc Loss = 397.43, KL Divergence = 205.70, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 3.87 mins
Memory Use (GB):

...

Memory Use (GB): 1.3077507019042969
Epoch: 60 / 201, Batch: 17 (2304 / 5504), Elapsed time: 166.48 mins
Enc Loss = 102.38, KL Divergence = 210.68, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 166.54 mins
Memory Use (GB): 1.3198585510253906
Epoch: 60 / 201, Batch: 18 (2432 / 5504), Elapsed time: 166.54 mins
Enc Loss = 103.99, KL Divergence = 205.92, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 166.60 mins
Memory Use (GB): 1.351470947265625
Epoch: 60 / 201, Batch: 19 (2560 / 5504), Elapsed time: 166.60 mins
Enc Loss = 100.13, KL Divergence = 205.68, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 166.67 mins
Memory Use (GB): 1.3260002136230469
Epoch: 60 / 201, Batch: 20 (2688 / 5504), Elapsed time: 166.67 mins
Enc Loss = 103.88, KL Divergence = 209.34, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Los

Memory Use (GB): 1.4503135681152344
Epoch: 61 / 201, Batch: 5 (768 / 5504), Elapsed time: 168.44 mins
Enc Loss = 100.63, KL Divergence = 203.59, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 168.50 mins
Memory Use (GB): 1.3271026611328125
Epoch: 61 / 201, Batch: 6 (896 / 5504), Elapsed time: 168.50 mins
Enc Loss = 100.24, KL Divergence = 208.56, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 168.56 mins
Memory Use (GB): 1.2672119140625
Epoch: 61 / 201, Batch: 7 (1024 / 5504), Elapsed time: 168.57 mins
Enc Loss = 101.66, KL Divergence = 207.75, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 168.63 mins
Memory Use (GB): 1.4016075134277344
Epoch: 61 / 201, Batch: 8 (1152 / 5504), Elapsed time: 168.63 mins
Enc Loss = 108.11, KL Divergence = 210.21, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999

Memory Use (GB): 1.5032691955566406
Epoch: 61 / 201, Batch: 37 (4864 / 5504), Elapsed time: 170.46 mins
Enc Loss = 105.74, KL Divergence = 211.31, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 170.52 mins
Memory Use (GB): 1.4306907653808594
Epoch: 61 / 201, Batch: 38 (4992 / 5504), Elapsed time: 170.52 mins
Enc Loss = 101.34, KL Divergence = 206.32, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 170.59 mins
Memory Use (GB): 1.4565086364746094
Epoch: 61 / 201, Batch: 39 (5120 / 5504), Elapsed time: 170.59 mins
Enc Loss = 102.93, KL Divergence = 206.19, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 170.65 mins
Memory Use (GB): 1.4088401794433594
Epoch: 61 / 201, Batch: 40 (5248 / 5504), Elapsed time: 170.65 mins
Enc Loss = 99.84, KL Divergence = 205.73, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Los

Memory Use (GB): 1.3696556091308594
Epoch: 62 / 201, Batch: 25 (3328 / 5504), Elapsed time: 172.42 mins
Enc Loss = 99.31, KL Divergence = 208.93, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 172.49 mins
Memory Use (GB): 1.3981246948242188
Epoch: 62 / 201, Batch: 26 (3456 / 5504), Elapsed time: 172.49 mins
Enc Loss = 100.12, KL Divergence = 206.01, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 172.55 mins
Memory Use (GB): 1.4934310913085938
Epoch: 62 / 201, Batch: 27 (3584 / 5504), Elapsed time: 172.55 mins
Enc Loss = 99.96, KL Divergence = 205.30, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 172.61 mins
Memory Use (GB): 1.5547332763671875
Epoch: 62 / 201, Batch: 28 (3712 / 5504), Elapsed time: 172.61 mins
Enc Loss = 101.12, KL Divergence = 207.56, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss

Memory Use (GB): 1.4816818237304688
Epoch: 63 / 201, Batch: 13 (1792 / 5504), Elapsed time: 174.39 mins
Enc Loss = 97.36, KL Divergence = 204.62, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 174.45 mins
Memory Use (GB): 1.4317855834960938
Epoch: 63 / 201, Batch: 14 (1920 / 5504), Elapsed time: 174.46 mins
Enc Loss = 102.58, KL Divergence = 209.61, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 174.52 mins
Memory Use (GB): 1.3640518188476562
Epoch: 63 / 201, Batch: 15 (2048 / 5504), Elapsed time: 174.52 mins
Enc Loss = 103.28, KL Divergence = 204.76, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 174.58 mins
Memory Use (GB): 1.4933128356933594
Epoch: 63 / 201, Batch: 16 (2176 / 5504), Elapsed time: 174.58 mins
Enc Loss = 101.72, KL Divergence = 207.02, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Los